##  Testing baseline algorithms for toys and games dataset
### Train Data: https://drive.google.com/file/d/180ZI9OIdivkO0T-H1wki1-K514iYYK_n
### Test Data: https://drive.google.com/file/d/1cW5doBO4jpbLQfZwygKFhy0CSSf6U1MQ

In [ ]:
# imports

import os
import math
import json
import random
from dotenv import load_dotenv
from huggingface_hub import login
import matplotlib.pyplot as plt
import numpy as np
import pickle
from collections import Counter

In [ ]:
# More imports for our traditional machine learning
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [ ]:
# NLP related imports
from sklearn.feature_extraction.text import CountVectorizer
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess

In [ ]:
# Finally, more imports for more advanced machine learning
from sklearn.svm import LinearSVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from polire import IDW

In [ ]:
# Constants - used for printing to stdout in color

GREEN = "\033[92m"
YELLOW = "\033[93m"
RED = "\033[91m"
RESET = "\033[0m"
COLOR_MAP = {"red":RED, "orange": YELLOW, "green": GREEN}

In [ ]:
%matplotlib inline

In [ ]:
from typing import Optional
from transformers import AutoTokenizer
import re

BASE_MODEL = "meta-llama/Meta-Llama-3.1-8B"

MIN_TOKENS = 150 # Any less than this, and we don't have enough useful content
MAX_TOKENS = 160 # Truncate after this many tokens. Then after adding in prompt text, we will get to around 180 tokens

MIN_CHARS = 300
CEILING_CHARS = MAX_TOKENS * 7

class Item:
    """
    An Item is a cleaned, curated datapoint of a Product with a Price
    """
    
    tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, trust_remote_code=True)
    PREFIX = "Price is $"
    QUESTION = "How much does this cost to the nearest dollar?"
    REMOVALS = ['"Batteries Included?": "No"', '"Batteries Included?": "Yes"', '"Batteries Required?": "No"', '"Batteries Required?": "Yes"', "By Manufacturer", "Item", "Date First", "Package", ":", "Number of", "Best Sellers", "Number", "Product "]

    title: str
    price: float
    category: str
    token_count: int = 0
    details: Optional[str]
    prompt: Optional[str] = None
    include = False

    def __init__(self, data, price):
        self.title = data['title']
        self.price = price
        self.parse(data)

    def scrub_details(self):
        """
        Clean up the details string by removing common text that doesn't add value
        """
        details = self.details
        for remove in self.REMOVALS:
            details = details.replace(remove, "")
        return details

    def scrub(self, stuff):
        """
        Clean up the provided text by removing unnecessary characters and whitespace
        Also remove words that are 7+ chars and contain numbers, as these are likely irrelevant product numbers
        """
        stuff = re.sub(r'[:\[\]"{}【】\s]+', ' ', stuff).strip()
        stuff = stuff.replace(" ,", ",").replace(",,,",",").replace(",,",",")
        words = stuff.split(' ')
        select = [word for word in words if len(word)<7 or not any(char.isdigit() for char in word)]
        return " ".join(select)
    
    def parse(self, data):
        """
        Parse this datapoint and if it fits within the allowed Token range,
        then set include to True
        """
        contents = '\n'.join(data['description'])
        if contents:
            contents += '\n'
        features = '\n'.join(data['features'])
        if features:
            contents += features + '\n'
        self.details = data['details']
        if self.details:
            contents += self.scrub_details() + '\n'
        if len(contents) > MIN_CHARS:
            contents = contents[:CEILING_CHARS]
            text = f"{self.scrub(self.title)}\n{self.scrub(contents)}"
            tokens = self.tokenizer.encode(text, add_special_tokens=False)
            if len(tokens) > MIN_TOKENS:
                tokens = tokens[:MAX_TOKENS]
                text = self.tokenizer.decode(tokens)
                self.make_prompt(text)
                self.include = True

    def make_prompt(self, text):
        """
        Set the prompt instance variable to be a prompt appropriate for training
        """
        self.prompt = f"{self.QUESTION}\n\n{text}\n\n"
        self.prompt += f"{self.PREFIX}{str(round(self.price))}.00"
        self.token_count = len(self.tokenizer.encode(self.prompt, add_special_tokens=False))

    def test_prompt(self):
        """
        Return a prompt suitable for testing, with the actual price removed
        """
        return self.prompt.split(self.PREFIX)[0] + self.PREFIX

    def __repr__(self):
        """
        Return a String version of this Item
        """
        return f"<{self.title} = ${self.price}>"

In [ ]:
# Download files from link

with open('./train_lite.pkl', 'rb') as file:
    train = pickle.load(file)

with open('./test_lite.pkl', 'rb') as file:
    test = pickle.load(file)

In [ ]:
class Tester:

    def __init__(self, predictor, title=None, data=test, size=200):
        self.predictor = predictor
        self.data = data
        self.title = title or predictor.__name__.replace("_", " ").title()
        self.size = size
        self.guesses = []
        self.truths = []
        self.errors = []
        self.sles = []
        self.colors = []

    def color_for(self, error, truth):
        if error<5 or error/truth < 0.1:
            return "green"
        elif error<15 or error/truth < 0.3:
            return "orange"
        else:
            return "red"
    
    def run_datapoint(self, i):
        datapoint = self.data[i]
        guess = self.predictor(datapoint)
        truth = datapoint.price
        error = abs(guess - truth)
        log_error = math.log(truth+1) - math.log(guess+1)
        sle = log_error ** 2
        color = self.color_for(error, truth)
        title = datapoint.title if len(datapoint.title) <= 40 else datapoint.title[:40]+"..."
        self.guesses.append(guess)
        self.truths.append(truth)
        self.errors.append(error)
        self.sles.append(sle)
        self.colors.append(color)
        print(f"{COLOR_MAP[color]}{i+1}: Guess: ${guess:,.2f} Truth: ${truth:,.2f} Error: ${error:,.2f} SLE: {sle:,.2f} Item: {title}{RESET}")

    def chart(self, title):
        max_error = max(self.errors)
        plt.figure(figsize=(12, 8))
        max_val = max(max(self.truths), max(self.guesses))
        plt.plot([0, max_val], [0, max_val], color='deepskyblue', lw=2, alpha=0.6)
        plt.scatter(self.truths, self.guesses, s=3, c=self.colors)
        plt.xlabel('Ground Truth')
        plt.ylabel('Model Estimate')
        plt.xlim(0, max_val)
        plt.ylim(0, max_val)
        plt.title(title)
        plt.show()

    def report(self):
        average_error = sum(self.errors) / self.size
        rmsle = math.sqrt(sum(self.sles) / self.size)
        hits = sum(1 for color in self.colors if color=="green")
        title = f"{self.title} Error=${average_error:,.2f} RMSLE={rmsle:,.2f} Hits={hits/self.size*100:.1f}%"
        self.chart(title)

    def run(self):
        self.error = 0
        for i in range(self.size):
            self.run_datapoint(i)
        self.report()

    @classmethod
    def test(cls, function):
        cls(function).run()

In [ ]:
def random_pricer(item):
    return random.randrange(1,60)

In [ ]:
# Set the random seed

random.seed(42)

# Run our TestRunner
Tester.test(random_pricer)

In [ ]:
# That was fun!
# We can do better - here's another rather trivial model

training_prices = [item.price for item in train]
training_average = sum(training_prices) / len(training_prices)

def constant_pricer(item):
    return training_average

In [ ]:
# Run our constant predictor
Tester.test(constant_pricer)


In [ ]:
# Create a new "features" field on items, and populate it with json parsed from the details dict

for item in train:
    item.features = json.loads(item.details)
for item in test:
    item.features = json.loads(item.details)

# Look at one

In [ ]:
train[0].features.keys()

In [ ]:
# Look at 20 most common features in training set

feature_count = Counter()
for item in train:
    for f in item.features.keys():
        feature_count[f]+=1

feature_count.most_common(40)

In [ ]:
# Now some janky code to pluck out the Item Weight
# Don't worry too much about this: spoiler alert, it's not going to be much use in training!

def get_weight(item):
    weight_str = item.features.get('Item Weight')
    if weight_str:
        parts = weight_str.split(' ')
        amount = float(parts[0])
        unit = parts[1].lower()
        if unit=="pounds":
            return amount
        elif unit=="ounces":
            return amount / 16
        elif unit=="grams":
            return amount / 453.592
        elif unit=="milligrams":
            return amount / 453592
        elif unit=="kilograms":
            return amount / 0.453592
        elif unit=="hundredths" and parts[2].lower()=="pounds":
            return amount / 100
        else:
            print(weight_str)
    return None

In [ ]:
weights = [get_weight(t) for t in train]
weights = [w for w in weights if w]

In [ ]:
average_weight = sum(weights)/len(weights)
average_weight

In [ ]:
def get_weight_with_default(item):
    weight = get_weight(item)
    return weight or average_weight

In [ ]:
def get_rank(item):
    rank_dict = item.features.get("Best Sellers Rank")
    if rank_dict:
        ranks = rank_dict.values()
        return sum(ranks)/len(ranks)
    return None

In [ ]:
ranks = [get_rank(t) for t in train]
ranks = [r for r in ranks if r]
average_rank = sum(ranks)/len(ranks)
average_rank

In [ ]:
def get_rank_with_default(item):
    rank = get_rank(item)
    return rank or average_rank

In [ ]:
def get_text_length(item):
    return len(item.test_prompt())

In [ ]:
# investigate the brands

brands = Counter()
for t in train:
    brand = t.features.get("Brand")
    if brand:
        brands[brand]+=1

# Look at most common 40 brands

brands.most_common(40)

In [ ]:
TOP_ELECTRONICS_BRANDS = ["hp", "dell", "lenovo", "samsung", "asus", "sony", "canon", "apple", "intel"]
def is_top_electronics_brand(item):
    brand = item.features.get("Brand")
    return brand and brand.lower() in TOP_ELECTRONICS_BRANDS

In [ ]:
def get_features(item):
    return {
        "weight": get_weight_with_default(item),
        "rank": get_rank_with_default(item),
        "text_length": get_text_length(item),
        "is_top_electronics_brand": 1 if is_top_electronics_brand(item) else 0
    }

In [ ]:
# Look at features in a training item
get_features(train[0])

In [ ]:
# A utility function to convert our features into a pandas dataframe

def list_to_dataframe(items):
    features = [get_features(item) for item in items]
    df = pd.DataFrame(features)
    df['price'] = [item.price for item in items]
    return df

train_df = list_to_dataframe(train)
test_df = list_to_dataframe(test)

In [ ]:
# Traditional Linear Regression!

np.random.seed(42)

# Separate features and target
feature_columns = ['weight', 'rank', 'text_length', 'is_top_electronics_brand']

X_train = train_df[feature_columns]
y_train = train_df['price']
X_test = test_df[feature_columns]
y_test = test_df['price']

# Train a Linear Regression
model = LinearRegression()
model.fit(X_train, y_train)

for feature, coef in zip(feature_columns, model.coef_):
    print(f"{feature}: {coef}")
print(f"Intercept: {model.intercept_}")

# Predict the test set and evaluate
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R-squared Score: {r2}")

In [ ]:
# Function to predict price for a new item

def linear_regression_pricer(item):
    features = get_features(item)
    features_df = pd.DataFrame([features])
    return model.predict(features_df)[0]

In [ ]:
# test it

Tester.test(linear_regression_pricer)

In [ ]:
# For the next few models, we prepare our documents and prices
# Note that we use the test prompt for the documents, otherwise we'll reveal the answer!!

prices = np.array([float(item.price) for item in train])
documents = [item.test_prompt() for item in train]

In [ ]:
# Use the CountVectorizer for a Bag of Words model

np.random.seed(42)
vectorizer = CountVectorizer(max_features=10000, stop_words='english')
X = vectorizer.fit_transform(documents)
regressor = LinearRegression()
regressor.fit(X, prices)

In [ ]:
def bow_lr_pricer(item):
    x = vectorizer.transform([item.test_prompt()])
    return max(regressor.predict(x)[0], 0)
pred_lr = {}
for i in range(len(test)):
    pred_lr[test[i]] = bow_lr_pricer(test[i])

In [ ]:
# test it
def get_pred_lr(item):
    return pred_lr[item]
Tester.test(get_pred_lr)

In [ ]:
xgb = GradientBoostingRegressor()
xgb.fit(X, prices)

In [ ]:
def bow_xgb_pricer(item):
    x = vectorizer.transform([item.test_prompt()])
    return max(xgb.predict(x)[0], 0)

In [ ]:
# test
pred_xgb = {}
for i in range(len(test)):
    pred_xgb[test[i]] = bow_xgb_pricer(test[i])
def get_pred_xgb(item):
    return pred_xgb[item]
Tester.test(get_pred_xgb)

In [ ]:
import spacy 
nlp = spacy.load("en_core_web_sm")

In [ ]:
def ner_doc(doc):
    d = nlp(doc.replace('\n',' '))
    ents = []
    for ent in d.ents:
        ents.extend(ent.text.split(' '))
    return ' '.join(list(set(ents)))
def ner_docs(docs):
    ret = []
    for i,doc in enumerate(docs):
        ret.append(ner_doc(doc))
        if i%1000 == 0:
            print(i, ret[-1])
    return ret

In [ ]:
if 0 and os.path.exists('docs.pkl'):
    docs2 = pickle.load(open('docs.pkl','rb'))
else:
    docs2 = ner_docs(documents)
    # pickle.dump(docs2, open('docs.pkl','wb'))

In [ ]:
np.random.seed(42)
vectorizer2 = CountVectorizer(max_features=10000, stop_words='english')
X2 = vectorizer2.fit_transform(docs2)
regressor2 = LinearRegression()
regressor2.fit(X2, prices)

In [ ]:
def ner_pricer(item):
    x = vectorizer2.transform([ner_doc(item.test_prompt())])
    return max(regressor2.predict(x)[0], 0)
# test
pred_ner = {}
for i in range(len(test)):
    pred_ner[test[i]] = ner_pricer(test[i])
def get_pred_ner(item):
    return pred_ner[item]
Tester.test(get_pred_ner)

In [ ]:
# The amazing word2vec model, implemented in gensim NLP library

np.random.seed(42)

# Preprocess the documents
processed_docs = [simple_preprocess(doc) for doc in documents]

# Train Word2Vec model
w2v_model = Word2Vec(sentences=processed_docs, vector_size=400, window=5, min_count=1, workers=8)

In [ ]:
# This step of averaging vectors across the document is a weakness in our approach

def document_vector(doc):
    doc_words = simple_preprocess(doc)
    word_vectors = [w2v_model.wv[word] for word in doc_words if word in w2v_model.wv]
    return np.mean(word_vectors, axis=0) if word_vectors else np.zeros(w2v_model.vector_size)

# Create feature matrix
X_w2v = np.array([document_vector(doc) for doc in documents])

In [ ]:
# Run Linear Regression on word2vec

word2vec_lr_regressor = LinearRegression()
word2vec_lr_regressor.fit(X_w2v, prices)

In [ ]:
def word2vec_lr_pricer(item):
    doc = item.test_prompt()
    doc_vector = document_vector(doc)
    return max(0, word2vec_lr_regressor.predict([doc_vector])[0])

In [ ]:
pred_lr_w2v = {}
for i in range(len(test)):
    pred_lr_w2v[test[i]] = word2vec_lr_pricer(test[i])
def get_pred_lr_w2v(item):
    return pred_lr_w2v[item]
Tester.test(get_pred_lr_w2v)

In [ ]:
# MIXTURE OF EXPERTS

In [ ]:
def get_pred_best2_mean(item):
    v1 = pred_lr[item]
    v2 = pred_lr_w2v[item]
    v3 = pred_xgb[item]
    d1 = abs(v1-v2)
    d2 = abs(v2-v3)
    d3 = abs(v3-v1)
    if d1 <= min(d2,d3):
        v = (v1+v2)/2
    elif d2 <= min(d1,d3):
        v = (v2+v3)/2
    else:
        v = (v1+v3)/2
    return v
Tester.test(get_pred_best2_mean)

In [ ]:
def get_pred_mean(item):
    v1 = pred_lr[item]
    v2 = pred_lr_w2v[item]
    v3 = pred_xgb[item]
    return (v1+v2+v3)/3
Tester.test(get_pred_mean)

In [ ]:
# Apply MinMax and Standard Scaler

In [ ]:
scalar = [MinMaxScaler, StandardScaler][1]().fit(X_w2v)
X_w2v_scaled = scalar.transform(X_w2v)

In [ ]:
word2vec_lr_reg_scaled = LinearRegression().fit(X_w2v_scaled, prices)

In [ ]:
def word2vec_lr_pricer_scaled(item):
    doc = item.test_prompt()
    doc_vector = document_vector(doc)
    doc_vector_scaled = scalar.transform([doc_vector])
    return max(0, word2vec_lr_reg_scaled.predict([doc_vector_scaled[0]])[0])

Tester.test(word2vec_lr_pricer_scaled)

In [ ]:
# Run XGB on word2vec

word2vec_xgb_regressor = GradientBoostingRegressor()
word2vec_xgb_regressor.fit(X_w2v, prices)

In [ ]:
def word2vec_xgb_pricer(item):
    doc = item.test_prompt()
    doc_vector = document_vector(doc)
    return max(0, word2vec_xgb_regressor.predict([doc_vector])[0])

In [ ]:
Tester.test(word2vec_xgb_pricer)

In [ ]:
# Support Vector Machines

np.random.seed(42)
svr_regressor = LinearSVR()

svr_regressor.fit(X_w2v, prices)

In [ ]:
def svr_pricer(item):
    np.random.seed(42)
    doc = item.test_prompt()
    doc_vector = document_vector(doc)
    return max(float(svr_regressor.predict([doc_vector])[0]),0)

In [ ]:
Tester.test(svr_pricer)

In [ ]:
# And the powerful Random Forest regression
mfile = 'random_forest_model.pkl'
if 0 and os.path.exists(mfile):
    rf_model = pickle.load(open(mfile,'rb'))
else:
    rf_model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=8)
    rf_model.fit(X_w2v, prices)

In [ ]:
def random_forest_pricer(item):
    doc = item.test_prompt()
    doc_vector = document_vector(doc)
    return max(0, rf_model.predict([doc_vector])[0])

In [ ]:
pred_rf = {}
for i in range(len(test)):
    pred_rf[test[i]] = random_forest_pricer(test[i])
def get_pred_rf(item):
    return pred_rf[item]
Tester.test(get_pred_rf)